# On-the-fly ggame computation

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import streamlit as st
import sklearn.metrics.pairwise as sklpw
from PIL import Image
from fuzzywuzzy import fuzz
import re

In [4]:
# LOAD DATA
allgamedata_df = pd.read_pickle('datasources/bgg_filters.pkl') # USE FOR URLS and FILTERS
allgamedata_df = allgamedata_df.astype({'game_rank':'int32'},copy=True)
allgamedocvects = np.load('datasources/allgamedocvects_v3.npz')['arr_0']
finalgamelist_df = pd.read_pickle('datasources/finalgamelist_df.pkl')
#finalgamelist_df =  finalgamelist_df.astype({'game_rank':'int32'},copy=True)
#finalgamelist_df['num_raters'] = [list(allgamedata_df.loc[allgamedata_df['game_name']==g,'num_raters'])[0] for g in finalgamelist_df['game_name']]
#finalgamelist_df.reset_index(drop=True,inplace=True) # So that row ids are indices to gamevector array

# FOr gameplay-based vectors
bgg_gameplay_df = pd.read_pickle('datasources/bgg_gameplayfeatures.pkl')
bgg_gameplay_df.dropna(inplace=True) # Some not-NAs here...
bgg_gameplay_df.reset_index(drop=True,inplace=True)
allgamePLAYdocvects = np.array(bgg_gameplay_df.iloc[:,1:]) # Create right here

## Verify data sizes

In [5]:
print('finalgamelist_df {} should match allgamedocvects {}'.format(finalgamelist_df.shape,allgamedocvects.shape))

finalgamelist_df (18561, 4) should match allgamedocvects (18561, 100)


In [6]:
print('bgg_gameplay_df {} should match allgamePLAYdocvects {}'.format(bgg_gameplay_df.shape,allgamePLAYdocvects.shape))

bgg_gameplay_df (18500, 272) should match allgamePLAYdocvects (18500, 271)


In [7]:
finalgamelist_df.columns

Index(['idx', 'game_rank', 'game_name', 'num_raters'], dtype='object')

In [ ]:
allgamedata_df.columns

In [ ]:
# FUNCTIONS

def getcompute_similar_by_gameplay(gamename,allgamedata_df, bgg_gameplay_df, allgamePLAYdocvects):
    # Get game rank from game name, and matrix index from rank
  
    gamerank = list(allgamedata_df.loc[allgamedata_df['game_name']==gamename,'game_rank'])[0]
    gamerank = int(gamerank)
    gamerank_idx = list(bgg_gameplay_df.index[bgg_gameplay_df['game_rank']==gamerank])[0]
    #print(gamename, gamerank_idx)
    
    mygamePLAYvector = allgamePLAYdocvects[gamerank_idx,:] 
    mygamePLAYvector = mygamePLAYvector.reshape(-1,1)

    mysimilarities_gp = []
    for t in range(0,allgamePLAYdocvects.shape[0]):
        currgamevect_gp = allgamePLAYdocvects[t,:]
        currgamevect_gp = currgamevect_gp.reshape(-1,1)
        dum = sklpw.cosine_similarity(currgamevect_gp.T,mygamePLAYvector.T)
        mysimilarities_gp.append(dum[0][0])
    mycompleteGPsimlist_df = pd.concat((pd.DataFrame({'game_rank':bgg_gameplay_df['game_rank']}),pd.DataFrame({'GameplaySimilarity':mysimilarities_gp})),axis=1)
    return mycompleteGPsimlist_df

def getcompute_similar_games(mygameid,mygamename,allgamedata_df,allgamedocvects,finalgamelist_df,bgg_gameplay_df, allgamePLAYdocvects,W1,W2,filt_dict):
    # Jun 13 added filt_dict input
    myvectid = mygameid
    mygamevector = allgamedocvects[myvectid,:]
    mygamevector= mygamevector.reshape(-1,1)
    mysimilarities = []
    for t in range(0,allgamedocvects.shape[0]):
        currgamevect = allgamedocvects[t,:]
        currgamevect = currgamevect.reshape(-1,1)
        dum = sklpw.cosine_similarity(currgamevect.T,mygamevector.T)
        mysimilarities.append(dum[0][0])
    mycompletesimlist_df = pd.concat((finalgamelist_df['game_rank'],finalgamelist_df['game_name'],pd.DataFrame({'Similarity':mysimilarities})),axis=1)
    #mycompletesimlist_df = pd.concat((finalgamelist_df['game_rank','game_name'],pd.DataFrame({'Similarity':mysimilarities})),axis=1)
    #Get this also for GAMEPLAY data
    mycompleteGPsimlist_df = getcompute_similar_by_gameplay(mygamename,allgamedata_df, bgg_gameplay_df, allgamePLAYdocvects)

    # PUt sim lists together
    mycompletesimlist_df  = mycompletesimlist_df.astype({'game_rank':'int32'},copy=True)
    mycompleteGPsimlist_df = mycompleteGPsimlist_df.astype({'game_rank':'int32'},copy=True)
    mycompletesimlist_df.set_index('game_rank',inplace=True)
    mycompleteGPsimlist_df.set_index('game_rank',inplace=True)
    # Do it
    myFINALsimlist_df = mycompletesimlist_df.join(mycompleteGPsimlist_df,how='inner')
    
    
    weightedsimilarity = (np.array(myFINALsimlist_df['Similarity'])*W1) + (np.array(myFINALsimlist_df['GameplaySimilarity'])*W2)
    myFINALsimlist_df['WghtdSimilarity'] = weightedsimilarity
    myFINALsimlist_df.sort_values(by='WghtdSimilarity',ascending=False,inplace=True)
    
    
    # NEW: Jun 13
    # Create DF with all the filter data
    dum=allgamedata_df.loc[[list(allgamedata_df.index[allgamedata_df['game_rank']==n])[0] for n in myFINALsimlist_df.index],:]
    dum = dum[[ 'avg_rating', 'bgg_url', 'numplayersmin', 'gamedurmin','agemin']].copy()
    dum.reset_index(drop=True, inplace=True)
    # Add this into myFINALsimlist_df
    myFINALsimlist_df.reset_index(inplace=True)
    myFINALsimlist_df=pd.concat((myFINALsimlist_df,dum),axis=1)
    # Create filter
    myfilters = (myFINALsimlist_df['avg_rating']>=filt_dict.get('min_rating'))  &  (myFINALsimlist_df['numplayersmin']>=filt_dict.get('min_players'))  &  (myFINALsimlist_df['gamedurmin']>=filt_dict.get('min_dur'))
    # UPDATE myFINALsimlist_df
    myFINALsimlist_df = myFINALsimlist_df.loc[myfilters,:].copy()
    
    # Create output list
    # MO: New Jun 13 - check to make sure there are enough games!
    if len(myFINALsimlist_df)>11:    
        mytop10simlist_df = myFINALsimlist_df[1:11].copy()
    else:
        mytop10simlist_df = myFINALsimlist_df.copy()
    urllist=[]
    for gamename in mytop10simlist_df['game_name']:
        urllist.append(list(allgamedata_df.loc[allgamedata_df['game_name']==gamename,'bgg_url'])[0])
    #mytop10simlist_df = pd.DataFrame({'Game':mytop10simlist_df['game_name'],'Similarity':mytop10simlist_df['WghtdSimilarity'],'url':urllist})
    mytop10simlist_df = pd.DataFrame({'Game':mytop10simlist_df['game_name'],'url':urllist})
    mytop10simlist_df.reset_index(drop=True,inplace=True)
    mytop10simlist_df.index = mytop10simlist_df.index+1

    return mytop10simlist_df,myFINALsimlist_df
    
    
    
    #mycompletesimlist_df.sort_values(by='Similarity',ascending=False,inplace=True)
    #mytop10simlist_df = mycompletesimlist_df[1:11]
    # Create output list
    #urllist=[]
    #for gamename in mytop10simlist_df['gamename']:
    #    urllist.append(list(allgamedata_df.loc[allgamedata_df['game_name']==gamename,'bgg_url'])[0])
    #mytop10simlist_df = pd.DataFrame({'Game':mytop10simlist_df['gamename'],'Similarity':mytop10simlist_df['Similarity'],'url':urllist})
    #mytop10simlist_df.reset_index(drop=True,inplace=True)
    #mytop10simlist_df.index = mytop10simlist_df.index+1
    #return mytop10simlist_df


def get_real_name_fuzzy(usergamename,finalgamelist_df):
    # Clean up
    usergamename = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", usergamename)
    #usergamename = re.sub(r"\d+", "", usergamename) # Maybe dont remove numbers?
    gamename_matchlist = [fuzz.token_sort_ratio(x,usergamename) for x in finalgamelist_df['game_name']]
    possiblegame_idx  = [i for i, x in enumerate(gamename_matchlist) if x == max(gamename_matchlist)]
    possiblegame_idx = possiblegame_idx[0] # Get first, make it number
    #possiblegame_name = list(finalgamelist_df.loc[finalgamelist_df['idx']==possiblegame_idx,'gamename'])[0]
    possiblegame_name = finalgamelist_df.iloc[possiblegame_idx,2]
    #print('Best match: {}'.format(possiblegame_name))
    
    return possiblegame_name,max(gamename_matchlist)

def make_clickable(url,text): # Make liks in pd df for url in table
    return f'<a target="_blank" href="{url}">{text}</a>'

def streamlitify_df(df):
    # Get original URLS
    df['Game_link'] = [make_clickable(a,b) for a,b in zip(list(df['url']),list(df['Game']))]
    return df


In [ ]:
# CREATES THE DEMO GAME LIST
allgamedata_df['numeric_ranks']=[int(x) for x in allgamedata_df['game_rank']]
topranked_df = pd.DataFrame(allgamedata_df.loc[allgamedata_df['numeric_ranks']<=50,'game_name']) # To go back n forth
topranked_df.sort_values(by='game_name',inplace=True)
topranked_idx = topranked_df.index
demo_gamelist = tuple(list(finalgamelist_df.loc[topranked_idx,'game_name']))

## Pretend some game selected

In [ ]:
mygamename = 'magic gather'
# IF NAME IS UNCLEAR
mygamename,qltynum = get_real_name_fuzzy(mygamename,finalgamelist_df)
print('Best guess: {} (match score: {}/100)'.format(mygamename,str(qltynum)))
mygameid = list(finalgamelist_df.index[finalgamelist_df['game_name']==mygamename])[0] # Need INDEX, not idx
mygameurl=list(allgamedata_df.loc[allgamedata_df['game_name']==mygamename,'bgg_url'])[0]

In [ ]:
W1=1 # Semantic
W2=1 # Feature
# FILTERS
filt_dict = {'min_rating':,'min_players':1,'min_dur':1, 'min_numraters':3}
#min_rating = 7


In [ ]:
mytop10simlist_df,myFINALsimlist_df = getcompute_similar_games(mygameid,mygamename,allgamedata_df,allgamedocvects,finalgamelist_df,bgg_gameplay_df, allgamePLAYdocvects,W1,W2,filt_dict)
mygamevect_df = streamlitify_df(mytop10simlist_df)
mygamevect_df

In [ ]:
mygamevect_df

In [ ]:
mygamevect_df

In [ ]:
mygamename_st_url = f'<a target="_blank" href="{mygameurl}">{mygamename}</a>'
mygamename_st_url

In [ ]:
mygameurl

#  ******** ROUGH STUFF ****************

In [ ]:
finalgamelist_df.columns
#finalgamelist_df[['game_rank','game_name']]

In [ ]:
myFINALsimlist_df = myFINALsimlist_df.iloc[0:10,:]
myFINALsimlist_df

In [ ]:
# Create DF with all the filter data
dum=allgamedata_df.loc[[list(allgamedata_df.index[allgamedata_df['game_rank']==n])[0] for n in myFINALsimlist_df.index],:]
dum = dum[[ 'avg_rating', 'bgg_url', 'numplayersmin', 'gamedurmin','agemin']].copy()
dum.reset_index(drop=True, inplace=True)
# Add this into myFINALsimlist_df
myFINALsimlist_df.reset_index(inplace=True)
myFINALsimlist_df=pd.concat((myFINALsimlist_df,dum),axis=1)
# Create filter
myfilters = (myFINALsimlist_df['avg_rating']>=filt_dict.get('min_rating'))  &  (myFINALsimlist_df['numplayersmin']>=filt_dict.get('min_players'))  &  (myFINALsimlist_df['gamedurmin']>=filt_dict.get('min_dur'))
# UPDATE myFINALsimlist_df
myFINALsimlist_df = myFINALsimlist_df.loc[myfilters,:].copy()

In [ ]:
mytop10simlist_df,myFINALsimlist_df = getcompute_similar_games(mygameid,mygamename,allgamedata_df,allgamedocvects,finalgamelist_df,bgg_gameplay_df, allgamePLAYdocvects,W1,W2,filt_dict)

In [ ]:
mytop10simlist_df

In [ ]:
urllist=[]
for gamename in mytop10simlist_df['game_name']:
    print(gamename)
    #urllist.append(list(allgamedata_df.loc[allgamedata_df['game_name']==gamename,'bgg_url'])[0])

In [ ]:
len(myFINALsimlist_df)

In [ ]:
allgamedata_df.columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk')

In [ ]:
plt.hist(np.log10(allgamedata_df['num_raters']))

In [ ]:
sns.jointplot(np.log10(allgamedata_df['num_raters']),allgamedata_df['avg_rating'],kind='hex')


In [ ]:
allgamedata_df.describe()
dum =allgamedata_df.copy()
dum['lognumraters'] = np.log10(allgamedata_df['num_raters'])

In [ ]:
sns.pairplot(dum[['game_rank','avg_rating','lognumraters','game_year','numplayersmin']])

In [ ]:
finalgamelist_df

In [ ]:
allgamedata_df.columns

In [ ]:
finalgamelist_df['numraters'] = [list(allgamedata_df.loc[allgamedata_df['game_name']==g,'num_raters'])[0] for g in finalgamelist_df['game_name']]
                                      
                                      
                                      

In [ ]:
finalgamelist_df

In [ ]:
finalgamelist_df.describe()